In [1]:
import os
import json

In [2]:
os.listdir('../github_structured_data/')

['characters.csv', 'graph.json', 'relations.csv']

# Info

In [12]:
wands = []
with open('../scrawl_fandom/wands.json') as f:
    wands = json.load(f)

In [18]:
wands_revised = {}
for wand in wands:
    owner = wand['wand_owner']
    wand_info = {}
    wand_info['wood'] = wand['wand_wood']
    wand_info['length'] = wand['wand_length']
    wand_info['core'] = wand['wand_core']
    wand_info['flexibility'] = wand['wand_flexibility']
    
    wands_revised[owner] = wand_info

In [19]:
wands_key_lst = list(wands_revised.keys())

In [37]:
multi_wands_char = {'Harry Potter':{'wand #2':['Harry_Potter_and_the_Deathly_Hallows'],
                                    'wand #1': ['other']}, 
                    'Ron Weasley' :{'wand #1': ["Harry_Potter_and_the_Philosopher's_Stone", "Harry_Potter_and_the_Chamber_of_Secrets"],
                                    'wand #2': ['Harry_Potter_and_the_Prisoner_of_Azkaban', 'Harry_Potter_and_the_Goblet_of_Fire', 'Harry_Potter_and_the_Order_of_the_Phoenix'],
                                    'wand #3': ['Harry_Potter_and_the_Half-Blood_Prince', 'Harry_Potter_and_the_Deathly_Hallows']}, 
                    "Jacob's sibling": {'wand #1': ['other'],
                                        'wand #2': ['Harry_Potter_and_the_Prisoner_of_Azkaban', 'Harry_Potter_and_the_Goblet_of_Fire', 'Harry_Potter_and_the_Order_of_the_Phoenix', 'Harry_Potter_and_the_Half-Blood_Prince', 'Harry_Potter_and_the_Deathly_Hallows']}}

In [3]:
kaggle_data = []
with open('../kaggle_structured_data/characters.csv') as f:
    first_line = []
    first = True
    for line in f:
        lines = line[:-1].split(';')
        #print(lines)
        if first == True:
            first_line = lines
            first = False
            continue
        
        row = {}
        for i in range(len(lines)):
            row[first_line[i]] = lines[i]
        kaggle_data.append(row)

In [4]:
all_books = os.listdir('../wikidata/')

In [5]:
all_books

['Harry_Potter_and_the_Chamber_of_Secrets',
 'Harry_Potter_and_the_Deathly_Hallows',
 'Harry_Potter_and_the_Goblet_of_Fire',
 'Harry_Potter_and_the_Half-Blood_Prince',
 'Harry_Potter_and_the_Order_of_the_Phoenix',
 "Harry_Potter_and_the_Philosopher's_Stone",
 'Harry_Potter_and_the_Prisoner_of_Azkaban']

In [8]:
all_books_info_dict = {}

for book in all_books:
    curr_path = '../wikidata/' + book + '/data/info.json'
    
    with open(curr_path) as f:
        data = json.load(f)
    
    all_books_info_dict[book] = data

In [61]:
def get_att(val, key):
    try:
        return val[key]
    except KeyError:
        return 'DNE'

In [80]:
combined_info_dict = {}

for book, book_val in all_books_info_dict.items():
    char_info = {}
    curr_book_info = {}
    
    for char in book_val:
        name = char['characterLabel']
        gender = get_att(char, 'genderLabel')
        blood_status = get_att(char, 'blood_statusLabel')
        species =get_att(char, 'speciesLabel')
        DOB = get_att(char, 'DOBLabel')
        POB = get_att(char, 'POBLabel')
        eye_color = get_att(char, 'eye_colorLabel')
        hair_color = get_att(char, 'hair_colorLabel')
        education = get_att(char, 'educationLabel')
        employer = get_att(char, 'employerLabel')
        membership = get_att(char, 'membershipLabel')
        
        has_wand = False
        for key in wands_key_lst:
            if name in key:
                has_wand = True
                wand_key = name
                break
        
        if name in multi_wands_char.keys():
            wand_num = 0
            for key, val in multi_wands_char[name].items():
                if book in val:
                    wand_num = key
            if wand_num == 0:
                wand_num = 'wand #1'
            wand_key = name + ' (' + wand_num + ')'
        
        if has_wand:
            wand_info = wands_revised[wand_key]
        else:
            wand_info = None
        
        patronus = 'DNE'
        for person in kaggle_data:
            try:
                person_name = person['Name'].split(' ')
            except KeyError:
                continue
            curr_name = name.split(' ')
            
            check = True
            for piece in curr_name:
                if piece in person_name:
                    pass
                else:
                    check = False
                    
            if check:
                patronus = get_att(person, 'Patronus')
                break
        
        if name in curr_book_info.keys():
            if education not in curr_book_info[name]['education']:
                curr_book_info[name]['education'].append(education)
            if employer not in curr_book_info[name]['employer']:
                curr_book_info[name]['employer'].append(employer)                
            if membership not in curr_book_info[name]['membership']:
                curr_book_info[name]['membership'].append(membership)                
            if hair_color not in curr_book_info[name]['hair_color']:
                curr_book_info[name]['hair_color'].append(hair_color)     
        else:
            val = {}
            val['gender'] = gender
            val['blood_status'] = blood_status
            val['species'] = species
            val['eye_color'] = eye_color
            val['hair_color'] = [hair_color]
            val['DOB'] = DOB
            val['POB'] = POB
            val['education'] = [education]
            val['employer'] = [employer]
            val['membership'] = [membership]
            val['wand'] = wand_info

            curr_book_info[name] = val
    combined_info_dict[book] = curr_book_info

In [81]:
with open('combined_info.json', 'w') as f:
    json.dump(combined_info_dict, f)

In [82]:
for key, val in combined_info_dict.items():
    file = './info/' + key + '.json'
    
    with open(file, 'w') as f:
        json.dump(val, f)

# Relation

In [77]:
all_books_relation_dict = {}

for book in all_books:
    curr_path = '../wikidata/' + book + '/data/relation.json'
    
    with open(curr_path) as f:
        data = json.load(f)
    
    all_books_relation_dict[book] = data

In [83]:
combined_relation_dict = {}

for book, book_val in all_books_relation_dict.items():
    curr_book_relation = {}
    
    for char in book_val:
        name = get_att(char, 'characterLabel')
        father = get_att(char, 'fatherLabel')
        mother = get_att(char, 'motherLabel')
        spouse = get_att(char, 'spouseLabel')
        child = get_att(char, 'childLabel')
        sibling = get_att(char, 'siblingLabel')
        relative = get_att(char, 'relativeLabel')
        in_relationship = get_att(char, 'in_relationshipLabel')
        enemy = get_att(char, 'enemyLabel')
        kill_by = get_att(char, 'kill_by_Label')
        owner = get_att(char, 'ownerLabel')
        
        if name in curr_book_relation.keys():
            if spouse not in curr_book_relation[name]['spouse']:
                curr_book_relation[name]['spouse'].append(spouse)
            if child not in curr_book_relation[name]['child']:
                curr_book_relation[name]['child'].append(child)
            if sibling not in curr_book_relation[name]['sibling']:
                curr_book_relation[name]['sibling'].append(sibling)
            if relative not in curr_book_relation[name]['relative']:
                curr_book_relation[name]['relative'].append(relative)
            if in_relationship not in curr_book_relation[name]['in_relationship']:
                curr_book_relation[name]['in_relationship'].append(in_relationship)
            if enemy not in curr_book_relation[name]['enemy']:
                curr_book_relation[name]['enemy'].append(enemy)
        else:
            val = {}
            val['father'] = father
            val['mother'] = mother
            val['spouse'] = [spouse]
            val['child'] = [child]
            val['sibling'] = [sibling]
            val['relative'] = [relative]
            val['in_relationship'] = [in_relationship]
            val['enemy'] = [enemy]
            val['kill_by'] = kill_by
            val['owner'] = owner
            
            curr_book_relation[name] = val
    combined_relation_dict[book] = curr_book_relation

In [85]:
with open('combined_relation.json', 'w') as f:
    json.dump(combined_relation_dict, f)

In [86]:
for key, val in combined_relation_dict.items():
    file = './relation/' + key + '.json'
    
    with open(file, 'w') as f:
        json.dump(val, f)